---
# Spark Exploration Exercises
---

In [ ]:
import pyspark
from pyspark.sql.functions import *
import wrangle
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.ion()
# tweak the matplotlib style defaults
plt.rc("figure", figsize=(13, 7))
plt.rc("font", size=14)
plt.style.use("ggplot")

## 1.

How many different cases are there, by department?

## 2.

Does the percentage of cases that are late vary by department?

## 3.

On average, how late are the late cases by department?

## 4.

What is the service type that is the most late? Just for Parks & Rec?

## 5.

For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

## 6.

Does whether or not its a weekend matter for when a case is opened/closed?

## 7.

On average, how many cases are opened a day for the Customer Service department?

## 8.

Does the number of service requests for the solid waste department vary by day of the week?